In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from numpy.random import randn
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
label = ['Y']
data = pd.read_csv('train_final.csv') 
X = data.drop(columns = ['Id','Y'])
y = data[label] 


####



In [ ]:
only_zero_train = data[data['Y']== 0]
only_ones_train = data[data['Y']== 1]
n=948
X = only_ones_train.sample(n)
df = pd.concat([only_zero_train, X])    
df = df.sample(frac=1)

In [ ]:
y = df['Y'] 
real = df.drop(columns = ['Y'])

In [ ]:
X

,Y,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
12630,1,26390,1,1.7700,118343,1,0,117961,174445,1,0,1,6,120099,0.8422,21009,149210,270488,1,120097,1,1,1,4,1
14071,1,5044,1,1.7700,118291,1,0,118290,132803,1,8,1,1,119503,1.0000,4997,121108,117887,1,119502,1,4,1,625,1
12586,1,78319,1,1.7700,118004,2,0,118003,117913,1,1,1,1,117880,-1.4123,61244,117878,19721,1,117879,1,1,1,13,1
2548,1,100002,1,1.7700,118225,1,0,117961,117906,1,1,1,5,118322,1.0000,1012,119924,290919,1,118321,1,1,1,1,1
13832,1,36385,1,1.7700,118603,1,0,118602,125168,1,22,1,1,118570,0.8924,14695,118008,19721,1,118568,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713,1,42030,1,3.5400,118463,1,0,118315,117906,1,1,1,1,118322,-2.1273,28245,123089,290919,1,118321,1,1,1,4,1
14552,1,73471,1,1.7700,118052,1,0,117961,117906,1,0,1,24,117908,1.0000,7551,118867,290919,1,117905,1,1,1,11,1
5566,1,25989,1,5.3100,121786,1,0,121785,117906,1,0,1,73,117908,1.0000,4263,119142,290919,1,117905,1,1,1,3,1
6052,1,17813,1,1.7700,118300,1,0,117961,239170,1,1,1,1,125753,-0.7682,17815,119019,118870,1,125751,1,1,1,1,1


In [ ]:
only_zero_train

,Y,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
22,0,14353,1,1.7700,118011,1,0,117916,119324,1,0,1,2,119325,1.9645,57818,118810,19793,1,119323,1,1,1,33,1
30,0,73800,1,1.7700,120141,1,0,120140,261774,1,0,1,1,118749,-0.8545,3697,118933,118453,1,118747,1,1,1,1,1
46,0,111914,1,1.7700,118291,1,0,118290,122739,1,0,1,11,118425,-1.6400,5163,118597,118424,1,118422,1,1,1,100,1
49,0,22393,1,10.6200,118026,1,0,91261,117906,1,4,1,1,118322,-2.2580,94240,119507,290919,1,118321,1,1,1,1,1
84,0,4678,1,1.7700,118052,1,0,117961,117906,1,0,1,36,118322,-0.2784,54618,118992,290919,1,118321,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16300,0,80660,1,1.7700,118225,1,0,117961,118959,1,0,1,1,120774,0.0000,19986,125139,118960,1,120773,1,1,1,4,1
16302,0,4672,1,3.5400,117962,1,1,117961,4673,1,0,1,2,119348,-0.4229,52687,118910,4673,1,119346,1,1,1,27,1
16331,0,32641,1,10.6200,119281,1,0,119280,193644,1,0,1,2,126685,0.0000,51733,117945,292795,1,126684,1,1,1,27,1
16347,0,31975,1,1.7700,118991,1,0,118990,305581,1,0,1,1,122954,-2.9027,7581,215920,19793,2,122952,1,1,1,3,1


In [ ]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    
    return X, y

In [ ]:
# generate n real samples with class labels; We randomly select n samples from the real data
def generate_real_samples(n):
    X = data.sample(n)
    y = np.ones((n, 1))
    return X, y

In [ ]:
feature_number = 24
def define_generator(latent_dim, n_outputs=feature_number):
    model = Sequential()
    model.add(Dense(50, activation='relu',  kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(n_outputs, activation='linear'))
    return model

In [ ]:
generator1 = define_generator(10, feature_number)
generator1.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 50)                550       
_________________________________________________________________
dense_34 (Dense)             (None, 60)                3060      
_________________________________________________________________
dense_35 (Dense)             (None, 24)                1464      
Total params: 5,074
Trainable params: 5,074
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def define_discriminator(n_inputs=feature_number):
    model = Sequential()
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
discriminator1 = define_discriminator(9)
discriminator1.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 50)                500       
_________________________________________________________________
dense_40 (Dense)             (None, 60)                3060      
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 61        
Total params: 3,621
Trainable params: 3,621
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [ ]:
# create a line plot of loss for the gan and save to file
def plot_history(d_hist, g_hist):
    # plot loss
    plt.subplot(1, 1, 1)
    plt.plot(d_hist, label='d')
    plt.plot(g_hist, label='gen')
    plt.show()
    plt.close()

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=200):
    # determine half the size of one batch, for updating the  discriminator
    half_batch = int(n_batch / 2)
    d_history = []
    g_history = []
    # manually enumerate epochs
    for epoch in range(n_epochs):
    
      # prepare real samples
      x_real, y_real = generate_real_samples(half_batch)
      # prepare fake examples
      x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      # update discriminator
      d_loss_real, d_real_acc = d_model.train_on_batch(x_real, y_real)
      d_loss_fake, d_fake_acc = d_model.train_on_batch(x_fake, y_fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      # prepare points in latent space as input for the generator
      x_gan = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = np.ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss_fake = gan_model.train_on_batch(x_gan, y_gan)
      print('>%d, d1=%.3f, d2=%.3f d=%.3f g=%.3f' % (epoch+1, d_loss_real, d_loss_fake, d_loss,  g_loss_fake))
      d_history.append(d_loss)
      g_history.append(g_loss_fake)
    plot_history(d_history, g_history)
    g_model.save('trained_generated_model.h5')

In [ ]:
pip install tabgan

     |████████████████████████████████| 44 kB 1.6 MB/s 
     |████████████████████████████████| 82 kB 362 kB/s 


In [ ]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np
label = ['Y']
data = pd.read_csv('train_final.csv') 
data.drop(columns = ['Id'], inplace=True)
test = pd.read_csv('test_final.csv') 
test.drop(columns = ['Id'], inplace=True)

train = data.drop(columns = ['Y'])
target = data[label]
new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train, target, test)

In [ ]:
test

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,37733,1,1.77,118603,1,0,118602,118097,1,0,1,1,117888,2.453740,13881,117941,117887,1,117885,1,1,1,1,1
1,312129,1,3.54,118052,1,0,117961,290919,1,4,1,43,118322,-0.012317,14638,118992,290919,1,118321,1,1,1,7,1
2,24884,1,23.01,118300,1,0,117961,302830,1,0,1,1,128231,1.000000,770,119181,4673,1,128230,1,1,1,14,1
3,4674,1,1.77,119091,1,0,119062,118036,1,9,1,1,117908,1.000000,16752,143531,290919,1,117905,1,1,1,81,1
4,68725,1,3.54,118300,1,0,117961,171056,1,0,1,6,118639,-0.503250,4945,118360,118638,1,118636,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16380,971,1,1.77,118300,1,0,117961,310997,1,0,1,1,120346,1.000000,4864,123055,118424,1,120344,1,1,1,44,1
16381,105802,1,1.77,118327,1,0,117961,117906,1,0,1,1,118322,0.580688,32457,118320,290919,1,118321,1,1,1,4,1
16382,43096,1,3.54,117962,1,0,117961,118368,1,0,1,1,118486,1.000000,2014,117904,117887,1,118368,1,1,1,1,1
16383,31220,1,1.77,118386,1,0,117961,117906,1,1,1,1,118322,-2.374507,1249,118910,290919,1,118321,1,1,1,2,2


In [ ]:
new_train1

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,45443,1,1.77,118327,1,0,117961,169009,1,1,1,3,117973,1.000000,1652,118507,117887,1,179731,1,1,1,4586,3
1,45443,1,1.77,118327,1,0,117961,169009,1,1,1,3,117973,1.000000,1652,118507,117887,1,179731,1,1,1,4586,3
2,79091,1,26.55,118052,1,0,117961,117906,1,0,1,1,117908,1.280075,3868,122938,290919,1,117905,1,1,1,35369,1
3,79091,1,26.55,118052,1,0,117961,117906,1,0,1,1,117908,1.280075,3868,122938,290919,1,117905,1,1,1,35369,1
4,79091,1,26.55,118052,1,0,117961,117906,1,0,1,1,117908,1.280075,3868,122938,290919,1,117905,1,1,1,35369,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47958,33247,1,53.10,118052,1,0,117961,305057,1,0,1,913,123738,1.716526,15887,120356,118960,1,123737,1,1,1,1,1
47959,33247,1,53.10,118052,1,0,117961,305057,1,0,1,913,123738,1.716526,15887,120356,118960,1,123737,1,1,1,1,1
47960,33247,1,53.10,118052,1,0,117961,305057,1,0,1,913,123738,1.716526,15887,120356,118960,1,123737,1,1,1,1,1
47961,20280,1,3.54,118011,1,0,117916,163899,1,0,1,690,117973,1.980056,19132,118599,117887,1,179731,1,1,1,3,1


In [ ]:
train

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,25884,1,33.63,118596,1,0,118595,125738,1,3,1,2,121374,-2.266430,1945,118450,119184,1,121372,1,1,1,2,1
1,34346,1,10.62,118041,1,0,117902,130913,1,1,1,23,118943,-0.305612,15385,117945,292795,1,259173,1,1,1,1,1
2,34923,1,1.77,118327,1,0,117961,124402,1,2,1,1,118786,2.015561,7547,118933,290919,1,118784,1,1,1,1,1
3,80926,1,30.09,118300,1,0,117961,301218,1,0,1,1,118332,-3.172501,4933,118458,118331,1,307024,1,1,1,2,1
4,4674,1,1.77,119921,1,0,119920,302830,1,0,1,2,128231,0.573767,13836,142145,4673,1,128230,1,1,1,620,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16378,33328,1,1.77,118343,1,0,117961,117906,1,0,1,1,118322,1.000000,4533,118700,290919,1,118321,1,1,1,1,1
16379,19944,1,1.77,118225,1,0,117961,132097,1,0,1,20,132098,1.000000,4663,120323,119095,3,132096,1,1,1,8,1
16380,28359,1,3.54,118327,1,0,117961,174445,1,2,1,2,132694,1.806069,7822,118492,270488,1,132692,1,2,1,1,1
16381,7542,1,3.54,118343,1,0,117961,117906,1,0,1,1,118786,-1.245079,6257,120722,290919,1,118784,1,1,1,12,1


In [ ]:
new_target1

0        1
1        1
2        1
3        1
4        1
        ..
47958    1
47959    1
47960    1
47961    1
47962    1
Name: Y, Length: 47963, dtype: int64

In [ ]:
!pip install table_evaluator


  Created wheel for table-evaluator: filename=table_evaluator-1.2.2.post1-py3-none-any.whl size=17051 sha256=f7ff7b98b27324f59b598a0f9eeb2eb0dd7f44a017b77efe8b46b9e7ffd6d61a
  Stored in directory: /root/.cache/pip/wheels/45/c7/20/47a3f8b044d705f3c088d9577968a805dded4b83096fabd2ac
Successfully built table-evaluator


In [ ]:
from table_evaluator import load_data, TableEvaluator

new_train1['Y'] = new_target1
train['Y'] = target
table_evaluator = TableEvaluator(train, new_train1)
table_evaluator.evaluate(target_col='Y')


Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                                     f1_real  f1_fake  jaccard_similarity
index                                                                    
LogisticRegression_real_testset       0.9384   0.9384              1.0000
LogisticRegression_fake_testset       0.9417   0.9417              1.0000
RandomForestClassifier_real_testset   0.9457   0.9747              0.9322
RandomForestClassifier_fake_testset   0.9829   0.9756              0.9454
DecisionTreeClassifier_real_testset   0.9258   0.9625              0.8635
DecisionTreeClassifier_fake_testset   0.9844   0.9704              0.9293
MLPClassifier_real_testset            0.8959   0.9384              0.9052
MLPClassifier_fake_testset            0.9051   0.9423              0.9136

Miscellaneous results:
                                             Result
Column Correlation Distance RMSE             0.0110
Column Correlation distance MAE              0.0

{'1 - MAPE Estimator results': 0.9774508491788314,
 'Basic statistics': 0.9902983235454998,
 'Correlation column correlations': 0.8877154417542508,
 'Mean Correlation between fake and real columns': 0.9110331252662667,
 'Similarity Score': 0.9416244349362122}

In [ ]:
# size of the latent space
latent_dim = 10
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)


ValueError: ignored

In [ ]:
from keras.models import load_model
model =load_model('/content/trained_generated_model')


OSError: ignored

In [ ]:
latent_points = generate_latent_points(10, 750)
X = model.predict(latent_points)
data_fake = pd.DataFrame(data=X,  columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'])
data_fake.head()

In [ ]:
!pip install table_evaluator


In [ ]:
from table_evaluator import load_data, TableEvaluator
table_evaluator = TableEvaluator(data, data_fake)
table_evaluator.evaluate(target_col='Outcome')

In [ ]:
table_evaluator.visual_evaluation()
